#### This notebook is created for the purpose of doing a Capstone Project using Python for implementing Visualization and Machine Learning Algorithms

In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries imported.')

Libraries imported.


I have downloaded the New York Museum data from internet. Upload the JSON file and create a Dataframe with all the New York Museums data

In [2]:
with open('rows.json') as json_data:
    newyork_data = json.load(json_data)

The data thet we need are in the 'data' tag of the Json. Lets look at the format of the data for 1 museum

In [3]:
newyork_mus = newyork_data['data']
newyork_mus[0]

['row-7jyv~2dj7.6utz',
 '00000000-0000-0000-4AE4-2FBF98C98CB8',
 0,
 1450753171,
 None,
 1450753171,
 None,
 '{ }',
 'POINT (-74.01375579519738 40.703816216918035)',
 'Alexander Hamilton U.S. Custom House',
 '(212) 514-3700',
 'http://www.oldnycustomhouse.gov/',
 '1 Bowling Grn',
 '',
 'New York',
 '10004.0']

Lets get the required details from the above Json file

In [4]:
mus_name = []
lats = []
longs = []
mus_addr1 = []
mus_addr2 = []
mus_city = []
mus_zip = []
mus_tel = []
mus_url = []

for museum in newyork_mus:
    mus_name.append(museum[9])
    mus_tel.append(museum[10])
    mus_addr1.append(museum[12])
    mus_addr2.append(museum[13])
    mus_city.append(museum[14])
    ny_zip=float(museum[15])
    mus_zip.append(int(ny_zip))
    mus_url.append(museum[11])
    latlong=museum[8]
    latlong=latlong.strip('POINT (')
    latlong=latlong.strip(')')
    long,lat=latlong.split(' ')
    lats.append(float(lat))
    longs.append(float(long))

Create a Dataframe from the above values

In [5]:
nymuseum_df = pd.DataFrame()
nymuseum_df['Museum_name'] = mus_name
nymuseum_df['Latitude'] = lats
nymuseum_df['Longitude'] = longs
nymuseum_df['Museum_addr1'] = mus_addr1
nymuseum_df['Museum_addr2'] = mus_addr2
nymuseum_df['Museum_City'] = mus_city
nymuseum_df['Museum_Zip'] = mus_zip
nymuseum_df['Museum_Telephone'] = mus_tel
nymuseum_df['Museum_Link'] = mus_url

In [6]:
nymuseum_df.head()

,Museum_name,Latitude,Longitude,Museum_addr1,Museum_addr2,Museum_City,Museum_Zip,Museum_Telephone,Museum_Link
0,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,1 Bowling Grn,,New York,10004,(212) 514-3700,http://www.oldnycustomhouse.gov/
1,Alice Austen House Museum,40.615121,-74.063032,2 Hylan Blvd,,Staten Island,10305,(718) 816-4506,http://www.aliceausten.org/
2,American Academy of Arts and Letters,40.833854,-73.947298,633 W. 155th St.,,New York,10032,(212) 368-5900,http://www.artsandletters.org/
3,American Folk Art Museum,40.761625,-73.978103,45 West 53rd Street,,New York,10019,(212) 265-1040,http://www.folkartmuseum.org/
4,American Immigration History Center,40.699056,-74.039685,Ellis Island,,New York,0,(212) 363-3200,http://www.ellisisland.org/


Select only the museums from New York

In [7]:
museum_ny = nymuseum_df[nymuseum_df['Museum_City'] == 'New York']

There are few records with zip code '0'. Remove those records as they are not helpfull for analysis

In [8]:
museum_ny[museum_ny['Museum_Zip'] == 0]

,Museum_name,Latitude,Longitude,Museum_addr1,Museum_addr2,Museum_City,Museum_Zip,Museum_Telephone,Museum_Link
4,American Immigration History Center,40.699056,-74.039685,Ellis Island,,New York,0,(212) 363-3200,http://www.ellisisland.org/
32,Ellis Island Museum,40.698208,-74.041322,Ellis Island,,New York,0,(212) 363-3200,http://www.ellisisland.org/
98,Rose Museum at Carnegie Hall,40.765012,-73.979911,154 West 57th Street,,New York,0,(212) 903-9629,http://www.carnegiehall.org/textSite/the_basic...


In [9]:
museum_ny=museum_ny[museum_ny['Museum_Zip'] != 0]

In [10]:
museum_ny.head()

,Museum_name,Latitude,Longitude,Museum_addr1,Museum_addr2,Museum_City,Museum_Zip,Museum_Telephone,Museum_Link
0,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,1 Bowling Grn,,New York,10004,(212) 514-3700,http://www.oldnycustomhouse.gov/
2,American Academy of Arts and Letters,40.833854,-73.947298,633 W. 155th St.,,New York,10032,(212) 368-5900,http://www.artsandletters.org/
3,American Folk Art Museum,40.761625,-73.978103,45 West 53rd Street,,New York,10019,(212) 265-1040,http://www.folkartmuseum.org/
5,American Museum of Natural History,40.780826,-73.973648,Central Park West at 79th Street,,New York,10024,(212) 769-5100,http://www.amnh.org/
6,American Numismatic Society,40.723527,-74.007012,75 Varick St,11th Floor,New York,10013,(212) 571-4470,http://www.numismatics.org/


I have created a dataset with New York Zip codes/Neighborhood and average property rent at each zip code.
Import the dataset

In [11]:
NYrent_zip = pd.read_csv('NYrent_by_zip.csv')

In [12]:
NYrent_zip.head()

,Museum_Zip,NY_Neighborhood,NY_Avg_Realest_price
0,10001,Chelsea and Clinton,3872
1,10002,Lower East Side,4472
2,10003,Lower East Side,4472
3,10004,Lower Manhattan,4553
4,10005,Lower Manhattan,4553


In [13]:
address = 'Manhattan, New York'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York are 40.7900869, -73.9598295.


In [14]:
map_NYmuseum = folium.Map(location=[latitude, longitude], zoom_start=11.5)

# add markers to map
for lat, lng, muse in zip(museum_ny['Latitude'], museum_ny['Longitude'], museum_ny['Museum_name']):
    label = folium.Popup(muse)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7
    ).add_to(map_NYmuseum)
    
map_NYmuseum

Define Foursquare credentioals

In [15]:
CLIENT_ID = '2INEDWW4OHPQR2XGLFRR5M1EDRFGHVAVAURJIPVWKQE44QLP' # your Foursquare ID
CLIENT_SECRET = '5AJDMF0AHJFZWEP5OOY2BXCZJR3F05Y03FPJQEFZ5XPNCRIW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2INEDWW4OHPQR2XGLFRR5M1EDRFGHVAVAURJIPVWKQE44QLP
CLIENT_SECRET:5AJDMF0AHJFZWEP5OOY2BXCZJR3F05Y03FPJQEFZ5XPNCRIW


Get top 50 venues around each museum within 500 meters

In [16]:
LIMIT = 50
def getNearbyVenues(names, latitudes, longitudes, m_zip, radius=500):
    
    venues_list=[]
    for name, lat, lng, mzip in zip(names, latitudes, longitudes, m_zip):
 #       print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            mzip,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Museum_Name', 
                  'Museum Latitude', 
                  'Museum Longitude', 
                  'Museum_Zip',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
museum_venue = getNearbyVenues(names=museum_ny['Museum_name'],
                                   latitudes=museum_ny['Latitude'],
                                   longitudes=museum_ny['Longitude'],
                                   m_zip=museum_ny['Museum_Zip']
                                  )

Check the count of venues returned for each museum

In [18]:
museum_venue.head()

,Museum_Name,Museum Latitude,Museum Longitude,Museum_Zip,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Battery Park,40.703552,-74.015596,Park
1,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Fraunces Tavern,40.703526,-74.011395,Bar
2,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Naya Express,40.702565,-74.012168,Lebanese Restaurant
3,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Hilton Garden Inn NYC Financial Center/Manhatt...,40.702961,-74.012281,Hotel
4,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Bowling Green,40.704987,-74.013656,Park


Add the average real estate price/sq feet for each Museum based on Zipcode

In [19]:
museum_venue = pd.merge(museum_venue, NYrent_zip, left_on='Museum_Zip', right_on='Museum_Zip')

In [20]:
museum_venue.head()

,Museum_Name,Museum Latitude,Museum Longitude,Museum_Zip,Venue,Venue Latitude,Venue Longitude,Venue Category,NY_Neighborhood,NY_Avg_Realest_price
0,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Battery Park,40.703552,-74.015596,Park,Lower Manhattan,4553
1,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Fraunces Tavern,40.703526,-74.011395,Bar,Lower Manhattan,4553
2,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Naya Express,40.702565,-74.012168,Lebanese Restaurant,Lower Manhattan,4553
3,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Hilton Garden Inn NYC Financial Center/Manhatt...,40.702961,-74.012281,Hotel,Lower Manhattan,4553
4,Alexander Hamilton U.S. Custom House,40.703816,-74.013756,10004,Bowling Green,40.704987,-74.013656,Park,Lower Manhattan,4553


In [21]:
museum_venue.groupby('Museum_Name').count()

,Museum Latitude,Museum Longitude,Museum_Zip,Venue,Venue Latitude,Venue Longitude,Venue Category,NY_Neighborhood,NY_Avg_Realest_price
Museum_Name,,,,,,,,,
Alexander Hamilton U.S. Custom House,50,50,50,50,50,50,50,50,50
American Academy of Arts and Letters,43,43,43,43,43,43,43,43,43
American Folk Art Museum,50,50,50,50,50,50,50,50,50
American Jewish Historical Society,50,50,50,50,50,50,50,50,50
American Museum of Natural History,50,50,50,50,50,50,50,50,50
American Numismatic Society,50,50,50,50,50,50,50,50,50
American Sephardi Federation / Sephardic House,50,50,50,50,50,50,50,50,50
Americas Society,50,50,50,50,50,50,50,50,50
Anne Frank Center USA,50,50,50,50,50,50,50,50,50


In [22]:
print('There are {} uniques categories.'.format(len(museum_venue['Venue Category'].unique())))

There are 303 uniques categories.


Since the Venue Categories are non-numeric variables, they have to be changed to numeric so that it can be used with clustering algorithm

In [23]:
museum_onehot = pd.get_dummies(museum_venue[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
museum_onehot['Museum_Name'] = museum_venue['Museum_Name'] 
museum_onehot['NY_Avg_Realest_price'] = museum_venue['NY_Avg_Realest_price'] 

# move Museum name and Avg price columns to the first column
fixed_columns = [museum_onehot.columns[-2]] + [museum_onehot.columns[-1]] + list(museum_onehot.columns[:-2])
museum_onehot = museum_onehot[fixed_columns]

museum_onehot.head()

,Museum_Name,NY_Avg_Realest_price,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shrine,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,TV Station,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,

In [24]:
museum_onehot.shape

(4067, 305)

Group the above data by Museum and take mean frequency of each venue category

In [25]:
museum_grouped = museum_onehot.groupby('Museum_Name').mean().reset_index()
museum_grouped

,Museum_Name,NY_Avg_Realest_price,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Cafeteria,Café,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Church,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Historic Site,History Museum,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shrine,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soba Restaurant,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Street Art,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,TV Station,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,

Take each museum with top 5 most common venue category

In [26]:
num_top_venues = 5

for mus in museum_grouped['Museum_Name']:
    print("----"+mus+"----")
    temp = museum_grouped[museum_grouped['Museum_Name'] == mus].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alexander Hamilton U.S. Custom House----
                  venue     freq
0  NY_Avg_Realest_price  4553.00
1          Cocktail Bar     0.08
2           Coffee Shop     0.08
3                  Park     0.06
4                   Bar     0.04


----American Academy of Arts and Letters----
                       venue     freq
0       NY_Avg_Realest_price  2200.00
1                       Park     0.09
2  Latin American Restaurant     0.07
3              Deli / Bodega     0.07
4       Fast Food Restaurant     0.05


----American Folk Art Museum----
                  venue     freq
0  NY_Avg_Realest_price  3872.00
1                 Hotel     0.08
2             Hotel Bar     0.06
3   American Restaurant     0.06
4            Steakhouse     0.04


----American Jewish Historical Society----
                           venue     freq
0           NY_Avg_Realest_price  3872.00
1           Gym / Fitness Center     0.08
2             Italian Restaurant     0.06
3  Vegetarian / Vegan Restaurant    

                  venue     freq
0  NY_Avg_Realest_price  3872.00
1     Korean Restaurant     0.34
2        Cosmetics Shop     0.04
3                 Hotel     0.04
4             Gastropub     0.04


----Leo Baeck Institute----
                  venue     freq
0  NY_Avg_Realest_price  3872.00
1  Gym / Fitness Center     0.08
2    Italian Restaurant     0.06
3          Cycle Studio     0.04
4                Bakery     0.04


----Madame Tussauds Wax Museum----
                  venue     freq
0  NY_Avg_Realest_price  3872.00
1               Theater     0.26
2          Burger Joint     0.06
3           Coffee Shop     0.06
4    Falafel Restaurant     0.04


----Merchant's House Museum----
                  venue     freq
0  NY_Avg_Realest_price  4472.00
1    Italian Restaurant     0.10
2                   Gym     0.06
3           Men's Store     0.04
4                  Café     0.04


----Metropolitan Museum of Art (MET)----
                  venue     freq
0  NY_Avg_Realest_price  4443.0

                  venue     freq
0  NY_Avg_Realest_price  4553.00
1                  Park     0.14
2    Mexican Restaurant     0.06
3                 Hotel     0.04
4                Museum     0.04


----Solomon R. Guggenheim Museum----
                  venue     freq
0  NY_Avg_Realest_price  4443.00
1    Italian Restaurant     0.10
2               Exhibit     0.10
3            Art Museum     0.08
4                  Café     0.04


----Sony Wonder Technology Lab----
                  venue     freq
0  NY_Avg_Realest_price  3950.00
1              Boutique     0.14
2         Jewelry Store     0.12
3                 Hotel     0.08
4                   Spa     0.08


----South Street Seaport Museum----
                  venue     freq
0  NY_Avg_Realest_price  4553.00
1                   Bar     0.06
2                  Café     0.06
3            Playground     0.04
4           Coffee Shop     0.04


----Studio Museum in Harlem----
                             venue     freq
0             NY

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a dataframe with top 10 venues for each Museum

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Museum_Name', 'Avg_price']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
museum_venues_sorted = pd.DataFrame(columns=columns)
museum_venues_sorted['Museum_Name'] = museum_grouped['Museum_Name']
museum_venues_sorted['Avg_price'] = museum_grouped['NY_Avg_Realest_price']

for ind in np.arange(museum_grouped.shape[0]):
    museum_venues_sorted.iloc[ind, 2:] = return_most_common_venues(museum_grouped.iloc[ind, 1:], num_top_venues)

museum_venues_sorted

,Museum_Name,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexander Hamilton U.S. Custom House,4553,Cocktail Bar,Coffee Shop,Park,Mexican Restaurant,Bar,Gym,Sandwich Place,Gym / Fitness Center,Steakhouse,Monument / Landmark
1,American Academy of Arts and Letters,2200,Park,Deli / Bodega,Latin American Restaurant,Grocery Store,Sandwich Place,Scenic Lookout,Mexican Restaurant,Chinese Restaurant,Pizza Place,Fast Food Restaurant
2,American Folk Art Museum,3872,Hotel,American Restaurant,Hotel Bar,Italian Restaurant,Boutique,Steakhouse,Art Museum,Museum,French Restaurant,Spa
3,American Jewish Historical Society,3872,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Women's Store,Gym,Bakery,Spa,Clothing Store,Japanese Restaurant,Speakeasy
4,American Museum of Natural History,4536,Exhibit,Park,History Museum,Planetarium,Theater,Italian Restaurant,Souvenir Shop,Garden,Bookstore,Sushi Restaurant
5,American Numismatic Society,5006,Italian Restaurant,Coffee Shop,Hotel,Cocktail Bar,Gym,Greek Restaurant,Optical Shop,Tea Room,Grocery Store,Roof Deck
6,American Sephardi Federation / Sephardic House,3872,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Women's Store,Bakery,Gym,Spa,Clothing Store,Japanese Restaurant,Speakeasy
7,Americas Society,4443,Boutique,Women's Store,French Restaurant,Italian Restaurant,Clothing Store,Art Museum,Gym / Fitness Center,Coffee Shop,Event Space,Tea Room
8,Anne Frank Center USA,5006,Clothing Store,Women's Store,Boutique,Café,Yoga Studio,Mediterranean Restaurant,French Restaurant,Men's Store,Furniture / Home Store,Swiss Restaurant
9,Asia Society,4443,Women's Store,Italian Restaurant,Art Museum,Boutique,Gym / Fitness Center,French Restaurant,Event Space,Coffee Shop,Steakhouse,Clothing Store


Now run K-means algorithm on the museum data to create clusters

In [29]:
kclusters = 5

museum_grouped_clustering = museum_grouped.drop(['Museum_Name'], axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(museum_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 2, 2, 0, 3, 2, 0, 3, 0])

In [30]:
museum_merge = museum_grouped.iloc[:,0:2]
museum_merge['Cluster Labels'] = kmeans.labels_
museum_merge = museum_merge.join(museum_ny.iloc[:,0:3].set_index('Museum_name'), on='Museum_Name')
museum_merge = museum_merge.join(museum_venues_sorted.set_index('Museum_Name'), on='Museum_Name') 
museum_merge = museum_merge.drop(['NY_Avg_Realest_price'], axis=1)
museum_merge

,Museum_Name,Cluster Labels,Latitude,Longitude,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexander Hamilton U.S. Custom House,0,40.703816,-74.013756,4553,Cocktail Bar,Coffee Shop,Park,Mexican Restaurant,Bar,Gym,Sandwich Place,Gym / Fitness Center,Steakhouse,Monument / Landmark
1,American Academy of Arts and Letters,1,40.833854,-73.947298,2200,Park,Deli / Bodega,Latin American Restaurant,Grocery Store,Sandwich Place,Scenic Lookout,Mexican Restaurant,Chinese Restaurant,Pizza Place,Fast Food Restaurant
2,American Folk Art Museum,2,40.761625,-73.978103,3872,Hotel,American Restaurant,Hotel Bar,Italian Restaurant,Boutique,Steakhouse,Art Museum,Museum,French Restaurant,Spa
3,American Jewish Historical Society,2,40.738015,-73.993929,3872,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Women's Store,Gym,Bakery,Spa,Clothing Store,Japanese Restaurant,Speakeasy
4,American Museum of Natural History,0,40.780826,-73.973648,4536,Exhibit,Park,History Museum,Planetarium,Theater,Italian Restaurant,Souvenir Shop,Garden,Bookstore,Sushi Restaurant
5,American Numismatic Society,3,40.723527,-74.007012,5006,Italian Restaurant,Coffee Shop,Hotel,Cocktail Bar,Gym,Greek Restaurant,Optical Shop,Tea Room,Grocery Store,Roof Deck
6,American Sephardi Federation / Sephardic House,2,40.738077,-73.993886,3872,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Women's Store,Bakery,Gym,Spa,Clothing Store,Japanese Restaurant,Speakeasy
7,Americas Society,0,40.768824,-73.965970,4443,Boutique,Women's Store,French Restaurant,Italian Restaurant,Clothing Store,Art Museum,Gym / Fitness Center,Coffee Shop,Event Space,Tea Room
8,Anne Frank Center USA,3,40.721127,-73.999630,5006,Clothing Store,Women's Store,Boutique,Café,Yoga Studio,Mediterranean Restaurant,French Restaurant,Men's Store,Furniture / Home Store,Swiss Restaurant
9,Asia Society,0,40.769834,-73.964284,4443,Women's Store,Italian Restaurant,Art Museum,Boutique,Gym / Fitness Center,French Restaurant,Event Space,Coffee Shop,Steakhouse,Clothing Store


Visualization of the clusters

In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(museum_merge['Latitude'], museum_merge['Longitude'], museum_merge['Museum_Name'], museum_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

As we can see there is high concentration of blue circles (cluster 2) in mid Manhattan. The red circles (Cluster 0) are concentrated around upper and lower Manhattan. Lets examine each cluster

Cluster 1

In [32]:
museum_merge.loc[museum_merge['Cluster Labels'] == 0, museum_merge.columns[[0] + [4] + list(range(5, museum_merge.shape[1]))]].head()

,Museum_Name,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alexander Hamilton U.S. Custom House,4553,Cocktail Bar,Coffee Shop,Park,Mexican Restaurant,Bar,Gym,Sandwich Place,Gym / Fitness Center,Steakhouse,Monument / Landmark
4,American Museum of Natural History,4536,Exhibit,Park,History Museum,Planetarium,Theater,Italian Restaurant,Souvenir Shop,Garden,Bookstore,Sushi Restaurant
7,Americas Society,4443,Boutique,Women's Store,French Restaurant,Italian Restaurant,Clothing Store,Art Museum,Gym / Fitness Center,Coffee Shop,Event Space,Tea Room
9,Asia Society,4443,Women's Store,Italian Restaurant,Art Museum,Boutique,Gym / Fitness Center,French Restaurant,Event Space,Coffee Shop,Steakhouse,Clothing Store
13,Children's Museum of Manhattan,4536,American Restaurant,Pub,Ice Cream Shop,Bookstore,Bar,Cosmetics Shop,Middle Eastern Restaurant,Burger Joint,Café,Coffee Shop


Cluster 2

In [33]:
museum_merge.loc[museum_merge['Cluster Labels'] == 1, museum_merge.columns[[0] + [4] + list(range(5, museum_merge.shape[1]))]].head()

,Museum_Name,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,American Academy of Arts and Letters,2200,Park,Deli / Bodega,Latin American Restaurant,Grocery Store,Sandwich Place,Scenic Lookout,Mexican Restaurant,Chinese Restaurant,Pizza Place,Fast Food Restaurant
10,Audubon Terrace,2200,Park,Deli / Bodega,Latin American Restaurant,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place,Donut Shop
15,Cloisters,2200,Deli / Bodega,Latin American Restaurant,Wine Bar,Spanish Restaurant,Art Museum,Coffee Shop,Bakery,Bank,Bar,Shoe Store
19,Dia at the Hispanic Society,2200,Deli / Bodega,Park,Grocery Store,Latin American Restaurant,Bar,Fast Food Restaurant,Chinese Restaurant,Mexican Restaurant,Pizza Place,Sandwich Place
21,Dyckman Farmhouse,2200,Café,Bakery,Wine Bar,Latin American Restaurant,Spanish Restaurant,Restaurant,American Restaurant,Deli / Bodega,Seafood Restaurant,Chinese Restaurant


Cluster 3

In [37]:
museum_merge.loc[museum_merge['Cluster Labels'] == 2, museum_merge.columns[[0] + [4] + list(range(5, museum_merge.shape[1]))]]

,Museum_Name,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,American Folk Art Museum,3872,Hotel,American Restaurant,Hotel Bar,Italian Restaurant,Boutique,Steakhouse,Art Museum,Museum,French Restaurant,Spa
3,American Jewish Historical Society,3872,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Women's Store,Gym,Bakery,Spa,Clothing Store,Japanese Restaurant,Speakeasy
6,American Sephardi Federation / Sephardic House,3872,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Women's Store,Bakery,Gym,Spa,Clothing Store,Japanese Restaurant,Speakeasy
11,Center For Jewish History,3872,Gym / Fitness Center,Italian Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Spa,Clothing Store,Arts & Crafts Store,Gym,Bakery,Women's Store
12,Chelsea Art Museum,3872,Art Gallery,Park,Coffee Shop,French Restaurant,Tapas Restaurant,Gift Shop,Golf Driving Range,Clothing Store,Bookstore,Gym / Fitness Center
17,Dahesh Museum,3950,Boutique,Jewelry Store,Spa,Hotel,Art Gallery,Hotel Bar,Leather Goods Store,Steakhouse,Indie Theater,Shoe Store
18,Dia Art Foundation,3872,Art Gallery,Park,Coffee Shop,French Restaurant,Hotel,Clothing Store,Bookstore,Chinese Restaurant,Boutique,Harbor / Marina
29,International Center of Photography,3872,Theater,Hotel,Food Truck,Coffee Shop,Bookstore,Concert Hall,Bar,Cuban Restaurant,French Restaurant,Clothing Store
30,Intrepid Sea-Air-Space Museum,3872,Gym,Boat or Ferry,Dog Run,Nightclub,Exhibit,Event Space,Seafood Restaurant,Strip Club,Bowling Alley,Gym / Fitness Center
35,John J. Harvey Fireboat,3872,Art Gallery,Park,Harbor / Marina,Cocktail Bar,Coffee Shop,Playground,Sandwich Place,Lounge,Gym,Event Space


Cluster 4

In [35]:
museum_merge.loc[museum_merge['Cluster Labels'] == 3, museum_merge.columns[[0] + [4] + list(range(5, museum_merge.shape[1]))]].head()

,Museum_Name,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,American Numismatic Society,5006,Italian Restaurant,Coffee Shop,Hotel,Cocktail Bar,Gym,Greek Restaurant,Optical Shop,Tea Room,Grocery Store,Roof Deck
8,Anne Frank Center USA,5006,Clothing Store,Women's Store,Boutique,Café,Yoga Studio,Mediterranean Restaurant,French Restaurant,Men's Store,Furniture / Home Store,Swiss Restaurant
14,Children's Museum of the Arts,5006,Café,Clothing Store,Women's Store,Boutique,Mediterranean Restaurant,Yoga Studio,Seafood Restaurant,French Restaurant,Wine Bar,Salon / Barbershop
20,Drawing Center,5006,Women's Store,Clothing Store,Men's Store,Boutique,Italian Restaurant,Shoe Store,Coffee Shop,Art Gallery,Bakery,Dessert Shop
22,East Village History Project,5006,Italian Restaurant,Bookstore,American Restaurant,Art Gallery,Gift Shop,Gym / Fitness Center,Boxing Gym,Sporting Goods Shop,Spa,Pilates Studio


Cluster 5

In [36]:
museum_merge.loc[museum_merge['Cluster Labels'] == 4, museum_merge.columns[[0] + [4] + list(range(5, museum_merge.shape[1]))]].head()

,Museum_Name,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,El Museo del Barrio,2528,Plaza,Grocery Store,Scenic Lookout,Café,Art Museum,Bridge,Field,Baseball Field,Beer Bar,Mexican Restaurant
54,Museum of the City of New York,2528,Thai Restaurant,Mexican Restaurant,Plaza,Café,Scenic Lookout,Grocery Store,Sandwich Place,Art Museum,Creperie,Park
69,Schomburg Center for Research in Black Culture,2783,African Restaurant,Café,Gym / Fitness Center,French Restaurant,Cosmetics Shop,Juice Bar,Public Art,Bookstore,Cafeteria,Caribbean Restaurant
75,Studio Museum in Harlem,2783,Southern / Soul Food Restaurant,Burger Joint,Italian Restaurant,French Restaurant,Lounge,Arts & Crafts Store,Cosmetics Shop,Jazz Club,Theater,Spanish Restaurant
78,The Jazz Museum in Harlem,2528,Theater,Burger Joint,Bus Stop,Grocery Store,Bus Station,Trail,Motorcycle Shop,Caribbean Restaurant,Chinese Restaurant,Park


In [38]:
museum_merge.loc[museum_merge['Museum_Name'] == 'Museum of Modern Art (MoMA)']

,Museum_Name,Cluster Labels,Latitude,Longitude,Avg_price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Museum of Modern Art (MoMA),2,40.761187,-73.977004,3872,Boutique,Hotel,American Restaurant,Jewelry Store,Hotel Bar,French Restaurant,Gift Shop,Spa,Gym,Art Museum
